In [3]:
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
%autoreload 2

import sys,os,glob
import psutil

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from pprint import pprint
from skimage import measure
from scipy.optimize import curve_fit

from tqdm import tqdm

from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs
from datetime import datetime as dt
import pickle

import datetime as dt
import re
import gc
import matplotlib.animation as animation
import warnings

## current script object
thismodule = sys.modules[__name__]

# Own settings
from settings import *

workdir = os.getcwd()
moduledir, fcndir = defineDir(workdir)

Own modules available: ['casestudy', 'conditionalstats', 'conditionalstats_chunked', 'lifecycle_mapping', 'plot1D', 'plot2D', 'plotCondPDFs']
Own modules available: ['PrecipGrid', 'fcns_load_DYAMOND_SAM', 'fcns_load_RCEMIP_SAM', 'fcns_movie', 'fcns_multiscale_extremes', 'load_TOOCAN_DYAMOND', 'load_TOOCAN_DYAMOND_modif', 'load_TOOCAN_DYAMOND_modif_BF', 'fcns_load_DYAMOND_SAM_Tb_Feng', 'load_TOOCAN_v208_DYAMOND', 'fcns_load_MCSMIP']


# Example relation table

In [6]:
#-- load own libraries

# to access segmentation files and simulation outputs
from fcns_load_DYAMOND_SAM import *

In [7]:
relation_table = loadRelTable('DYAMOND_SEG') 

In [8]:
relation_table

,Unnamed: 0,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,img_seg_path,str_code,Unnamed: 0.1,Unnamed: 0.1.1,year,month,day,hour,minute,path_dyamond,UTC
0,476,476.0,476.0,476.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-1-30,2.0,2.0,2016.0,8.0,1.0,1.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000000720,17014.03
1,443,443.0,443.0,443.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-2-0,3.0,3.0,2016.0,8.0,1.0,2.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000000960,17014.04
2,468,468.0,468.0,468.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-2-30,4.0,4.0,2016.0,8.0,1.0,2.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000001200,17014.05
3,432,432.0,432.0,432.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-3-0,5.0,5.0,2016.0,8.0,1.0,3.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000001440,17014.06
4,437,437.0,437.0,437.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-3-30,6.0,6.0,2016.0,8.0,1.0,3.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000001680,17014.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,1295,1295.0,1295.0,1295.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-9-22-30,1916.0,1916.0,2016.0,9.0,9.0,22.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460080,17053.45
1915,1315,1315.0,1315.0,1315.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-9-23-0,1917.0,1917.0,2016.0,9.0,9.0,23.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460320,17053.46
1916,1313,1313.0,1313.0,1313.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-9-23-30,1918.0,1918.0,2016.0,9.0,9.0,23.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460560,17053.47
1917,1304,1304.0,1304.0,1304.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-10-0-0,1919.0,1919.0,2016.0,9.0,10.0,0.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460800,17053.48


In [21]:
relation_table['img_seg_path'][0]

'data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/2016_08_01/ImageSegTOOCAN_20160801-03.nc'

# Construct a relation table for other model and other TOOCAN version

In [149]:
model ='FV3'
toocan_version = 'v2.07'
toocan_version_str = toocan_version.replace('.','')
DIR_DYAMOND = DIR_ROOTS['MCSMIP']['model_output']%model
DIR_TOOCANSEG = DIR_ROOTS['MCSMIP']['TOOCANSEG']%(model,toocan_version)
dt_min = dt.datetime.strptime(BOUNDS['MCSMIP']['model_output'][model]['start'],"%Y%m%d%H")
dt_max = dt.datetime.strptime(BOUNDS['MCSMIP']['model_output'][model]['end'],"%Y%m%d%H")

In [150]:
DIR_DYAMOND

'/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Summer/FV3/olr_pcp_instantaneous'

In [151]:
! ls /bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Summer/FV3/olr_pcp_instantaneous/pr_rlut_fv3_2016080223.nc

/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Summer/FV3/olr_pcp_instantaneous/pr_rlut_fv3_2016080223.nc


In [152]:
DIR_TOOCANSEG

'/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMOND/Summer/FV3/TOOCAN/TOOCAN_v2.07/GLOBAL/2016'

In [153]:
! ls /bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMOND/Summer/FV3/TOOCAN/TOOCAN_v2.07/GLOBAL/2016

2016_08_01  2016_08_13	2016_08_25  2016_09_06
2016_08_02  2016_08_14	2016_08_26  2016_09_07
2016_08_03  2016_08_15	2016_08_27  2016_09_08
2016_08_04  2016_08_16	2016_08_28  2016_09_09
2016_08_05  2016_08_17	2016_08_29  FileTracking
2016_08_06  2016_08_18	2016_08_30  FileTracking_NETCDF
2016_08_07  2016_08_19	2016_08_31  FinalFileTracking
2016_08_08  2016_08_20	2016_09_01  NewFinalFileTracking
2016_08_09  2016_08_21	2016_09_02  TOOCAN-GLOBAL-20160801-20160831.dat.gz
2016_08_10  2016_08_22	2016_09_03  TOOCAN-GLOBAL-20160901-20160909.dat.gz
2016_08_11  2016_08_23	2016_09_04
2016_08_12  2016_08_24	2016_09_05


In [154]:
! ls /bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMOND/Summer/FV3/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/2016_08_02

mcs_mask_TOOCAN_FV3_20160802-001.nc  mcs_mask_TOOCAN_FV3_20160802-013.nc
mcs_mask_TOOCAN_FV3_20160802-002.nc  mcs_mask_TOOCAN_FV3_20160802-014.nc
mcs_mask_TOOCAN_FV3_20160802-003.nc  mcs_mask_TOOCAN_FV3_20160802-015.nc
mcs_mask_TOOCAN_FV3_20160802-004.nc  mcs_mask_TOOCAN_FV3_20160802-016.nc
mcs_mask_TOOCAN_FV3_20160802-005.nc  mcs_mask_TOOCAN_FV3_20160802-017.nc
mcs_mask_TOOCAN_FV3_20160802-006.nc  mcs_mask_TOOCAN_FV3_20160802-018.nc
mcs_mask_TOOCAN_FV3_20160802-007.nc  mcs_mask_TOOCAN_FV3_20160802-019.nc
mcs_mask_TOOCAN_FV3_20160802-008.nc  mcs_mask_TOOCAN_FV3_20160802-020.nc
mcs_mask_TOOCAN_FV3_20160802-009.nc  mcs_mask_TOOCAN_FV3_20160802-021.nc
mcs_mask_TOOCAN_FV3_20160802-010.nc  mcs_mask_TOOCAN_FV3_20160802-022.nc
mcs_mask_TOOCAN_FV3_20160802-011.nc  mcs_mask_TOOCAN_FV3_20160802-023.nc
mcs_mask_TOOCAN_FV3_20160802-012.nc  mcs_mask_TOOCAN_FV3_20160802-024.nc


*******

Check correspondence between model output and MCS masks:
    
    pr_rlut_fv3_2016080223.nc corresponds to mcs_mask_TOOCAN_FV3_20160802-024.nc
    
so there is an offset in the hour index

*******

In [155]:
# initialize table columns

str_code_all = []
img_seg_path_all = []
path_dyamond_all = []
UTC_all = []

In [156]:
# iterate over hours between early July and mid-September

dt_0 = dt.datetime(year=2016,month=8,day=1,hour=0)
dt_delta = dt.timedelta(seconds=3600)

for i_t in range(0,960):
    
    dt_current = dt_0 + i_t*dt_delta

    if dt_current >= dt_min and dt_current <= dt_max:
        
        # print(dt_current.strftime("%Y%m%d%H"),end='; ')

        #- compute info
        # date
        date_str = dt_current.strftime("%Y%m%d%H")
        # segmentation mask path
        date_str_2 = dt_current.strftime("%Y_%m_%d")
        year = dt_current.strftime("%Y")
        filename = FILE_ROOTS['MCSMIP']['TOOCANSEG']%(model,dt_current.strftime("%Y%m%d")+"-"+str(dt_current.hour+1).rjust(3,'0'))
        img_seg_path = os.path.join(DIR_TOOCANSEG,date_str_2,filename)
        # output path
        filename = FILE_ROOTS['MCSMIP']['model_output'][model]%date_str
        path_dyamond = os.path.join(DIR_DYAMOND,filename)
        # UTC <days from 1970-01-01>.<hour>
        UTC = "%d."%((dt.datetime(year=2016,month=8,day=1)-dt.datetime(year=1970,month=1,day=1)).days)+str(dt_current.hour).rjust(2,'0')

        #- store info
        str_code_all.append(date_str) # STOPPED HERE
        img_seg_path_all.append(img_seg_path)
        path_dyamond_all.append(path_dyamond)
        UTC_all.append(UTC)

    else:

        print('step %d outside bounds'%i_t)
        str_code_all.append(None)
        img_seg_path_all.append(None)
        path_dyamond_all.append(None)
        UTC_all.append(None)

#- Store all of these in a pandas dataframe

d = {'date': str_code_all,
     'path_toocanseg': img_seg_path_all,
     'path_dyamond': path_dyamond_all,
     'UTC': UTC_all}


relation_table = pd.DataFrame(data=d)

step 0 outside bounds


In [157]:
relation_table

,date,path_toocanseg,path_dyamond,UTC
0,None,None,None,None
1,2016080101,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.01
2,2016080102,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.02
3,2016080103,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.03
4,2016080104,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.04
...,...,...,...,...
955,2016090919,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.19
956,2016090920,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.20
957,2016090921,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.21
958,2016090922,/bdd/MT_WORKSPACE/lgouttes/MODELS/MCSMIP/DYAMO...,/bdd/MT_WORKSPACE/MCS/MODELS/MCSMIP/DYAMOND/Su...,17014.22


In [112]:
#- Save
file_out = os.path.join(DIR_DATA,'relation_table_UTC_dyamond_segmentation_%s_%s.csv'%(model,toocan_version_str))
relation_table.to_csv(file_out)

# Automate for all models

In [143]:

def buildRelationTable(model,toocan_version):
    
    #- load paths
    toocan_version_str = toocan_version.replace('.','')
    DIR_DYAMOND = DIR_ROOTS['MCSMIP']['model_output']%model
    DIR_TOOCANSEG = DIR_ROOTS['MCSMIP']['TOOCANSEG']%(model,toocan_version)
    dt_min = dt.datetime.strptime(BOUNDS['MCSMIP']['model_output'][model]['start'],"%Y%m%d%H")
    dt_max = dt.datetime.strptime(BOUNDS['MCSMIP']['model_output'][model]['end'],"%Y%m%d%H")
    
    #- initialize table columns

    str_code_all = []
    img_seg_path_all = []
    path_dyamond_all = []
    UTC_all = []

    #- iterate over hours between early July and mid-September

    dt_0 = dt.datetime(year=2016,month=8,day=1,hour=0)
    dt_delta = dt.timedelta(seconds=3600)

    for i_t in range(0,960):

        dt_current = dt_0 + i_t*dt_delta

        if dt_current >= dt_min and dt_current <= dt_max:

            # print(dt_current.strftime("%Y%m%d%H"),end='; ')

            #- compute info
            # date
            date_str = dt_current.strftime("%Y%m%d%H")
            # segmentation mask path
            date_str_2 = dt_current.strftime("%Y_%m_%d")
            year = dt_current.strftime("%Y")
            filename = FILE_ROOTS['MCSMIP']['TOOCANSEG']%(model,dt_current.strftime("%Y%m%d")+"-"+str(dt_current.hour+1).rjust(3,'0'))
            img_seg_path = os.path.join(DIR_TOOCANSEG,date_str_2,filename)
            # output path
            filename = FILE_ROOTS['MCSMIP']['model_output'][model]%date_str
            path_dyamond = os.path.join(DIR_DYAMOND,filename)
            # UTC <days from 1970-01-01>.<hour>
            UTC = "%d."%((dt.datetime(year=2016,month=8,day=1)-dt.datetime(year=1970,month=1,day=1)).days)+str(dt_current.hour).rjust(2,'0')
            
            #- store info
            str_code_all.append(date_str) # STOPPED HERE
            img_seg_path_all.append(img_seg_path)
            path_dyamond_all.append(path_dyamond)
            UTC_all.append(UTC)

        else:

            print('step %d outside bounds'%i_t)
            str_code_all.append(None)
            img_seg_path_all.append(None)
            path_dyamond_all.append(None)
            UTC_all.append(None)
            
    #- Store all of these in a pandas dataframe

    d = {'date': str_code_all,
         'path_toocanseg': img_seg_path_all,
         'path_dyamond': path_dyamond_all,
         'UTC': UTC_all}

    relation_table = pd.DataFrame(data=d)

    #- Save
    file_out = os.path.join(DIR_DATA,'relation_table_UTC_dyamond_segmentation_%s_%s.csv'%(model,toocan_version_str))
    relation_table.to_csv(file_out)
    
    print('%s-%s complete'%(model,toocan_version))
    
    return

In [144]:
models = 'ARPEGE', 'FV3', 'IFS', 'MPAS', 'NICAM', 'OBS', 'SAM', 'UM'
toocan_version = 'v2.07'

for model in models:
    
    buildRelationTable(model,toocan_version)

step 0 outside bounds
step 1 outside bounds
step 2 outside bounds
step 3 outside bounds
step 4 outside bounds
step 5 outside bounds
step 6 outside bounds
step 7 outside bounds
step 8 outside bounds
step 9 outside bounds
step 10 outside bounds
step 11 outside bounds
step 12 outside bounds
step 13 outside bounds
step 14 outside bounds
step 15 outside bounds
step 16 outside bounds
step 17 outside bounds
step 18 outside bounds
step 19 outside bounds
step 20 outside bounds
step 21 outside bounds
step 22 outside bounds
step 23 outside bounds
ARPEGE-v2.07 complete
step 0 outside bounds
FV3-v2.07 complete
IFS-v2.07 complete
step 0 outside bounds
MPAS-v2.07 complete
NICAM-v2.07 complete
OBS-v2.07 complete
step 0 outside bounds
SAM-v2.07 complete
UM-v2.07 complete
